In [1]:
import os
import cv2
from deepface import DeepFace
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
import pickle


In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def cosine_distance(a, b):
    return 1 - cosine_similarity(a, b)

In [3]:
persons = []
 
for r, d, f in os.walk("../sources/faces/"): # r=root, d=directories, f = file
 for file in f:
    exact_path = r + "/" + file
    persons.append(exact_path)

representations = []
for person in persons:
  representation = DeepFace.represent(img_path = person, model_name = "ArcFace")[0]["embedding"]

  instance = []
  instance.append(person)
  instance.append(representation)
  representations.append(instance)

f = open("../sources/faces/representations.pkl", "wb")
pickle.dump(representations, f)
f.close()

ValueError: Exception while loading ../sources/faces//representations.pkl

In [4]:
with open('../sources/metadata/image_database.json', 'r') as f:
    image_database = json.load(f)

In [5]:
f = open("../sources/faces//representations.pkl", "rb")
representations = pickle.load(f)

results = []
for image_info in tqdm(image_database, desc="Processing Images"):
  try:
    if image_info['path'].split("\\")[-2] == "camera":
      image_path = os.path.join(image_info['path'])

      face_objs = DeepFace.extract_faces(img_path=image_path, enforce_detection=False)
      image_info['total_faces'] = len(face_objs)

      persons = []
      for i, face_obj in enumerate(face_objs):
        if face_obj['confidence'] > 0.9:

          face_img = (face_obj['face'] * 255).astype('uint8')
          cv2.imwrite("../sources/temp/current_face.jpg", cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))
          face_representation = DeepFace.represent(img_path="../sources/temp/current_face.jpg", model_name="ArcFace", enforce_detection=False)[0]["embedding"]
          distances = []
          for i in range(0, len(representations)):
            source_name = representations[i][0]
            source_representation = representations[i][1]
            distance = cosine_distance(source_representation, face_representation)
            distances.append(distance)

          idx = np.argmin(distances)
          matched_name = representations[idx][0].split("/")[-2]
          persons.append(matched_name)     
      image_info["faces_label"]=list(dict.fromkeys(persons))
    else:
      image_info['total_faces'] = 0
      image_info["faces_label"] = []
  except Exception as e:
    print(f"Error extracting face from {image_path} : {str(e)}")
    
    


Processing Images: 100%|██████████| 6140/6140 [23:50<00:00,  4.29it/s]  


In [6]:
with open('../sources/metadata/image_database.json', 'w') as f:
    json.dump(image_database, f, indent=4)